## Naive Approach
Take a list of cities, select those that are "between" my friend and me (see "bounding box" below), sort by population and select the most relevant, calculate train travel times from both of us to each city, select the city where the **one of us arriving latest arrives as early as possible** (metric 1). 

## Obvious Limitations & Ideas For Improvement
- distance matrix API function seems to have an upper limit for locations as input => we need to pre-filter locations to likely 
- stations or cities? 
  - if cities instead of stations used, this will cause inefficiency because the "middle" from the city dataset might be far away from the most easily accessible station in that city) 
  - stations have the disadvantage that it's harder to filter by population / connectedness => we should merge the two datasets below (e.g. by assigning each station to the city it's in)
- what do we mean by "minimising travelling effort"? 
  - distance matrix API seems to provide durations only! This means we're **not actually** minimising latest arrival in (1) but longer travel time => current fix is to generate top 5 solutions and compare arrival times  

In [1]:
import os 
import gmaps, googlemaps
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
client = googlemaps.Client(key=os.environ["GOOGLE_DISTMAT_APIKEY"])
gmaps.configure(api_key=os.environ["GOOGLE_DISTMAT_APIKEY"])

In [3]:
def coords(place_name):  # don't run this too often — free queries are limited
    geocoded = client.geocode(place_name)
    if len(geocoded) > 1:
        raise RuntimeError(f"Multiple places found for query '{place_name}': " + str(geocoded))
    if len(geocoded) == 0: 
        raise RunetimeError(f"No places found for query '{place_name}'!")
    return list(geocoded[0]["geometry"]["location"].values())

In [4]:
def bounding_box(locations, padding=0.01):  # expects list of coordinate tuples
    x = [l[0] for l in locations]; y = [l[1] for l in locations]
    return [(min(x)-padding, min(y)-padding), [max(x)+padding, min(y)-padding], 
            (max(x)+padding, max(y)+padding), [min(x)-padding, max(y)+padding]]

def in_box(loc, box):  # expects loc = [x,y], box = [bot left, bot right, top right, top left]
    return box[0][0] <= loc[0] <= box[2][0] and box[0][1] <= loc[1] <= box[2][1]

In [5]:
me = "Bonn Hbf"; friend = "Amsterdam Centraal"
loc1 = coords(me); loc2 = coords(friend)

In [57]:
time_start = datetime(2023,11,25,8,30,0)
time_start.isoformat()

'2023-11-25T08:30:00'

In [58]:
box = bounding_box([loc1,loc2], 0.25)

European stations dataset from [Kaggle](https://www.kaggle.com/datasets/headsortails/train-stations-in-europe/)

In [59]:
all_stations = pd.read_csv("train_stations_europe.csv")
relevant_stations = all_stations[all_stations.apply(lambda x: in_box([x["latitude"],x["longitude"]], box),axis=1)]
station_locs = relevant_stations[["latitude","longitude"]].to_numpy()

World cities with more than 1000 inhabitants from [opendatasoft.com](https://public.opendatasoft.com/explore/dataset/geonames-all-cities-with-a-population-1000/export/?disjunctive.cou_name_en&sort=name)

In [60]:
all_cities = pd.read_csv("all_cities.csv",sep=";")
relevant_cities = all_cities[(all_cities["Country Code"] == "DE") | (all_cities["Country Code"] == "NL")]
relevant_cities[["lat","lon"]] = relevant_cities["Coordinates"].str.split(", ", expand=True).astype(float)
relevant_cities = relevant_cities[relevant_cities.apply(lambda x: in_box([x["lat"],x["lon"]],box),axis=1)]
relevant_cities = relevant_cities.sort_values(by="Population", ascending=False)  # TODO: don't hardcode  

/var/folders/n4/1s67j3n10s74cqwhy7mxgbhr0000gn/T/ipykernel_14128/2056482694.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relevant_cities[["lat","lon"]] = relevant_cities["Coordinates"].str.split(", ", expand=True).astype(float)
/var/folders/n4/1s67j3n10s74cqwhy7mxgbhr0000gn/T/ipykernel_14128/2056482694.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relevant_cities[["lat","lon"]] = relevant_cities["Coordinates"].str.split(", ", expand=True).astype(float)


In [61]:
relevant_cities = relevant_cities[:250]  # select top k by population 
city_locs = relevant_cities[["lat","lon"]].to_numpy()
relevant_cities[["Name","Population","lat","lon"]]

,Name,Population,lat,lon
2796,Köln,963395,50.93333,6.95000
56162,Amsterdam,741636,52.37403,4.88969
89389,Düsseldorf,620523,51.22172,6.77616
132330,Essen,593085,51.45657,7.01228
2845,Duisburg,504358,51.43247,6.76516
...,...,...,...,...
40601,Wülfrath,21003,51.28195,7.03821
10862,Geertruidenberg,20941,51.70167,4.85694
3143,Leerdam,20758,51.89333,5.09167
20129,Bockum,20617,51.34696,6.61589


Visualisation of all cities considered:

In [62]:
fig = gmaps.figure()
fig.add_layer(gmaps.marker_layer([loc1,loc2]))
#fig.add_layer(gmaps.symbol_layer(station_locs[:],fill_color='blue'))
fig.add_layer(gmaps.symbol_layer(city_locs[:],fill_color='blue'))
fig.add_layer(gmaps.drawing_layer(features=[
    gmaps.Polygon(box, stroke_color="blue", fill_color='blue')]))
fig

Figure(layout=FigureLayout(height='420px'))

Get distances from friend's and my location to every relevant city (**warning:** can cause many API calls, potentially limit city_locs before) 

In [63]:
chunk = 25  # seems to be maximum allowed (API error if larger)
dist_mat_list = [client.distance_matrix([loc1,loc2], locations, departure_time=time_start, mode="transit") 
                 for locations in np.array_split(city_locs, np.ceil(len(city_locs) / chunk))]

In [64]:
columns = np.concatenate([dist_mat["destination_addresses"] for dist_mat in dist_mat_list])
time_mat = np.concatenate([np.array([[route["duration"]["value"] if route["status"] == "OK" else np.inf
      for route in row["elements"]] for row in dist_mat["rows"]]) for dist_mat in dist_mat_list], axis=1)
time_texts = np.concatenate([np.array([[route["duration"]["text"] if route["status"] == "OK" else "impossible"
      for route in row["elements"]] for row in dist_mat["rows"]]) for dist_mat in dist_mat_list], axis=1)

Calculate best location using metric (1) from above (find 'num_res' best options):

In [65]:
num_res = 10
idx_k_best = np.argpartition(np.max(time_mat,axis=0), num_res)[:num_res]  
addrs_best = columns[idx_k_best]

In [66]:
time_texts[:,idx_k_best].T

array([['1 hour 35 mins', '2 hours 45 mins'],
       ['2 hours 24 mins', '2 hours 45 mins'],
       ['1 hour 45 mins', '2 hours 41 mins'],
       ['2 hours 42 mins', '2 hours 35 mins'],
       ['2 hours 51 mins', '2 hours 46 mins'],
       ['2 hours 21 mins', '2 hours 24 mins'],
       ['2 hours 34 mins', '2 hours 5 mins'],
       ['2 hours 52 mins', '2 hours 27 mins'],
       ['1 hour 57 mins', '2 hours 58 mins'],
       ['1 hour 21 mins', '2 hours 54 mins']], dtype='<U15')

In [67]:
locs_best = [coords(addr) for addr in addrs_best]
my_routes = [client.directions(me, addr, departure_time=time_start, mode="transit") for addr in addrs_best]
friends_routes = [client.directions(friend, addr, departure_time=time_start, mode="transit") for addr in addrs_best]

**Sanity check:** distmat optimal duration should equal directions duration for the same time/location. As we can see, this is not the case. Unfortunately, it's hard to tell where the difference comes from without knowing what exactly the API does internally. 

In [68]:
durations = np.array([[my_routes[i][0]["legs"][0]["duration"]["text"], 
           friends_routes[i][0]["legs"][0]["duration"]["text"], 
           my_routes[i][0]["legs"][-1]["arrival_time"]["text"].encode("ascii","ignore").decode(),
           friends_routes[i][0]["legs"][-1]["arrival_time"]["text"].encode("ascii","ignore").decode()] 
        for i in range(num_res)])
durations

array([['1 hour 36 mins', '2 hours 45 mins', '10:06AM', '11:53AM'],
       ['2 hours 23 mins', '2 hours 41 mins', '11:25AM', '11:25AM'],
       ['1 hour 45 mins', '2 hours 41 mins', '10:29AM', '11:49AM'],
       ['2 hours 42 mins', '2 hours 32 mins', '11:44AM', '11:16AM'],
       ['2 hours 51 mins', '2 hours 44 mins', '11:53AM', '11:18AM'],
       ['2 hours 21 mins', '2 hours 22 mins', '11:05AM', '11:06AM'],
       ['2 hours 34 mins', '2 hours 2 mins', '11:36AM', '10:37AM'],
       ['2 hours 52 mins', '2 hours 24 mins', '11:54AM', '10:59AM'],
       ['1 hour 56 mins', '2 hours 55 mins', '10:40AM', '11:39AM'],
       ['1 hour 21 mins', '2 hours 54 mins', '10:05AM', '12:01PM']],
      dtype='<U15')

**Note**: how even when durations are similar, arrival times can differ extremely! (see 'limitations' above). We now choose the best route by earliest arrival time.

In [69]:
arrival_times = np.array([[my_routes[i][0]["legs"][-1]["arrival_time"]["value"], 
                           friends_routes[i][0]["legs"][-1]["arrival_time"]["value"]] for i in range(num_res)])
idx_best = np.argmin(np.max(arrival_times,axis=1))
loc_best = locs_best[idx_best]; addr_best = addrs_best[idx_best]
my_route = my_routes[idx_best]; friends_route = friends_routes[idx_best]
idx_best, addr_best

(5, 'Oude Markt 2, 5911 HG Venlo, Netherlands')

In [70]:
def route_steps(route):
    res = []
    for step in route["steps"]:
        if "transit_details" in step:
            d = step["transit_details"]
            line = d["line"]["name"] if "name" in d["line"] else d["line"]["short_name"]
            t1 = d["departure_time"]["text"].encode("ascii","ignore").decode()
            t2 = d["arrival_time"]["text"].encode("ascii","ignore").decode()
            res.append(f'Take the {line} ({d["headsign"]}) at {t1} to {d["arrival_stop"]["name"]}, arrive {t2}')
        else: 
            res.append(step["html_instructions"])
            
    return res

In [71]:
result = (time_start.isoformat(), route_steps(my_route[0]["legs"][0]), 
          route_steps(friends_route[0]["legs"][0]),list(durations[idx_best]))
result

('2023-11-25T08:30:00',
 ['Walk to Bonn Hbf',
  'Take the IC2441 (Dortmund Hbf) at 8:46AM to Düsseldorf Central Station, arrive 9:32AM',
  'Walk to Düsseldorf Central Station',
  'Take the RE13 (Venlo) at 9:48AM to Venlo, arrive 10:56AM',
  'Walk to Oude Markt 2, 5911 HG Venlo, Netherlands'],
 ['Walk to Amsterdam Centraal',
  'Take the Enkhuizen <-> Heerlen IC3900 (Heerlen) at 8:45AM to Utrecht Centraal, arrive 9:11AM',
  'Take the Schiphol Airport <-> Venlo IC3500 (Venlo) at 9:24AM to Venlo, arrive 10:55AM',
  'Walk to Oude Markt 2, 5911 HG Venlo, Netherlands'],
 ['2 hours 21 mins', '2 hours 22 mins', '11:05AM', '11:06AM'])

**TODO:** plot my_route and friends_route directly - they might differ from gmaps.directions.Diretions

In [73]:
fig = gmaps.figure(layout={"height":"600px"})
fig.add_layer(gmaps.marker_layer([loc1,loc2,loc_best]))
fig.add_layer(gmaps.directions.Directions(loc1, loc_best, departure_time=time_start, travel_mode='TRANSIT'))
fig.add_layer(gmaps.directions.Directions(loc2, loc_best, departure_time=time_start, travel_mode='TRANSIT'))
fig

Figure(layout=FigureLayout(height='600px'))